In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Laptop_Train_v2.xml
/kaggle/input/Restaurants_Test_Data_PhaseB.csv
/kaggle/input/laptops-trial.csv
/kaggle/input/Laptops_Test_Data_PhaseB.csv
/kaggle/input/restaurants-trial.xml
/kaggle/input/SemEvalSchema.xsd
/kaggle/input/Laptops_Test_Data_phaseB.xml
/kaggle/input/Laptops_Test_Data_PhaseA.xml
/kaggle/input/Laptops_Test_Data_PhaseA.csv
/kaggle/input/Restaurants_Train_v2.csv
/kaggle/input/laptops-trial.xml
/kaggle/input/Restaurants_Train_v2.xml
/kaggle/input/Restaurants_Test_Data_phaseB.xml
/kaggle/input/Restaurants_Test_Data_PhaseA.csv
/kaggle/input/Restaurants_Test_Data_PhaseA.xml
/kaggle/input/Restaurants_Train.xml
/kaggle/input/Laptop_Train_v2.csv
/kaggle/input/restaurants-trial.csv


In [2]:
import os

# Duyệt qua tất cả các file trong thư mục /kaggle/input
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'restaurants' in filename.lower():  
            print(os.path.join(dirname, filename))

/kaggle/input/Restaurants_Test_Data_PhaseB.csv
/kaggle/input/restaurants-trial.xml
/kaggle/input/Restaurants_Train_v2.csv
/kaggle/input/Restaurants_Train_v2.xml
/kaggle/input/Restaurants_Test_Data_phaseB.xml
/kaggle/input/Restaurants_Test_Data_PhaseA.csv
/kaggle/input/Restaurants_Test_Data_PhaseA.xml
/kaggle/input/Restaurants_Train.xml
/kaggle/input/restaurants-trial.csv


In [3]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/Restaurants_Train_v2.csv')
train_df

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,1634,"The food is uniformly exceptional, with a very...",kitchen,positive,55,62
4,1634,"The food is uniformly exceptional, with a very...",menu,neutral,141,145
...,...,...,...,...,...,...
3688,671,Each table has a pot of boiling water sunken i...,pot of boiling water,neutral,17,37
3689,671,Each table has a pot of boiling water sunken i...,meats,neutral,99,104
3690,671,Each table has a pot of boiling water sunken i...,vegetables,neutral,114,124
3691,671,Each table has a pot of boiling water sunken i...,rice,neutral,130,134


# Model

## Tiền Xử lí 

### file csv

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

2025-10-01 13:57:16.092870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759327036.264357      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759327036.314611      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from datasets import Dataset

In [6]:
from transformers import AutoTokenizer
from datasets import Dataset
tokenizer_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
def df_to_hf_dataset(df, tokenizer):
    
    bio_data = []
    grouped = df.groupby('id', sort=False)

    for _, group in grouped:
        sentence = group['Sentence'].iloc[0]
        
        aspects = [(int(row['from']), int(row['to'])) for _, row in group.iterrows()]
        encoding = tokenizer(sentence, return_offsets_mapping=True, truncation=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
        offsets = encoding['offset_mapping']
        labels = [0] * len(tokens)
        for start, end in aspects:
            for i, (s, e) in enumerate(offsets):
                if e > start and s < end:  
                    if s <= start:
                        labels[i] = 1
                    else:
                        labels[i] = 2

        bio_data.append({'tokens': tokens, 'labels': labels})
    dataset = Dataset.from_list(bio_data)
    return dataset


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
dataset = df_to_hf_dataset(train_df,tokenizer)

In [8]:
dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 2021
})

In [9]:
train_df

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,1634,"The food is uniformly exceptional, with a very...",kitchen,positive,55,62
4,1634,"The food is uniformly exceptional, with a very...",menu,neutral,141,145
...,...,...,...,...,...,...
3688,671,Each table has a pot of boiling water sunken i...,pot of boiling water,neutral,17,37
3689,671,Each table has a pot of boiling water sunken i...,meats,neutral,99,104
3690,671,Each table has a pot of boiling water sunken i...,vegetables,neutral,114,124
3691,671,Each table has a pot of boiling water sunken i...,rice,neutral,130,134


In [10]:
train_df['Sentence'][0]

'But the staff was so horrible to us.'

In [11]:
print(dataset['tokens'][0])

['[CLS]', 'but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', '.', '[SEP]']


In [12]:
print(dataset['labels'][0])

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [13]:
import torch

In [14]:
import torch

def prepare_dataset_for_bert(dataset, tokenizer):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for tokens, labels in zip(dataset['tokens'], dataset['labels']):
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        label_ids = labels.copy()
        # gán -100 cho [CLS] và [SEP]
        label_ids[0] = -100
        label_ids[-1] = -100

       
        input_ids_list.append(torch.tensor(input_ids))
        attention_mask_list.append(torch.tensor(attention_mask))
        labels_list.append(torch.tensor(label_ids))

    return input_ids_list, attention_mask_list, labels_list


In [15]:
from transformers import AutoTokenizer
input_ids_list, attention_mask_list, labels_list = prepare_dataset_for_bert(dataset, tokenizer)

In [16]:
print(dataset['tokens'][0])
print(len(dataset['tokens'][0]))

['[CLS]', 'but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', '.', '[SEP]']
11


In [17]:
# Kiểm tra
print(input_ids_list[0])
print(attention_mask_list[0])
print(labels_list[0])

print(len(input_ids_list[0]))
print(len(attention_mask_list[0]))
print(len(labels_list[0]))


tensor([ 101, 2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149, 1012,  102])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([-100,    0,    0,    1,    0,    0,    0,    0,    0,    0, -100])
11
11
11


In [18]:
hf_dataset = Dataset.from_dict({
    'input_ids': input_ids_list,
    'attention_mask': attention_mask_list,
    'labels': labels_list
})

## Tạo các tập

In [19]:
from transformers import AutoTokenizer
from datasets import Dataset
tokenizer_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [20]:
# /kaggle/input/Restaurants_Test_Data_PhaseB.csv
# /kaggle/input/restaurants-trial.xml
# /kaggle/input/Restaurants_Train_v2.csv
# /kaggle/input/Restaurants_Train_v2.xml
# /kaggle/input/Restaurants_Test_Data_phaseB.xml
# /kaggle/input/Restaurants_Test_Data_PhaseA.csv
# /kaggle/input/Restaurants_Test_Data_PhaseA.xml
# /kaggle/input/Restaurants_Train.xml
# /kaggle/input/restaurants-trial.csv
path_train = "/kaggle/input/Restaurants_Train_v2.csv"
path_trial = "/kaggle/input/restaurants-trial.csv"

In [21]:
import pandas as pd

df_train = pd.read_csv(path_train)
df_trial = pd.read_csv(path_trial)

df_all = pd.concat([df_train, df_trial], axis=0, ignore_index=True)

In [22]:
print(df_all.shape)

(3789, 6)


In [23]:
df_all

,id,Sentence,Aspect Term,polarity,from,to
0,3121,But the staff was so horrible to us.,staff,negative,8,13
1,2777,"To be completely fair, the only redeeming fact...",food,positive,57,61
2,1634,"The food is uniformly exceptional, with a very...",food,positive,4,8
3,1634,"The food is uniformly exceptional, with a very...",kitchen,positive,55,62
4,1634,"The food is uniformly exceptional, with a very...",menu,neutral,141,145
...,...,...,...,...,...,...
3784,2912,"The dishes offered were unique, very tasty and...",sardines with biscuits,positive,77,99
3785,2912,"The dishes offered were unique, very tasty and...",large whole shrimp,positive,101,119
3786,2912,"The dishes offered were unique, very tasty and...",pistachio ice cream,positive,135,154
3787,3188,Went there for an office lunch.,office lunch,neutral,18,30


In [24]:
all_train = df_to_hf_dataset(df_all,tokenizer)
input_ids_list_train, attention_mask_list_train, labels_list_train = prepare_dataset_for_bert(all_train, tokenizer)

all_dataset = Dataset.from_dict({
    'input_ids': input_ids_list_train,
    'attention_mask': attention_mask_list_train,
    'labels': labels_list_train
})

In [25]:
all_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2021
})

In [26]:
from datasets import Dataset, DatasetDict

train_testvalid = all_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

dataset_dict = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1616
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 202
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 203
    })
})


## Class_weight 

In [27]:
from collections import Counter
import torch

all_labels = [label for seq in dataset_dict['train']['labels'] for label in seq if label != -100]

In [28]:
counts = Counter(all_labels)
print(counts)

Counter({0: 25500, 1: 2961, 2: 2295})


In [29]:
total = sum(counts.values())

class_weights = {label: total / count for label, count in counts.items()}
print(class_weights)


{0: 1.2061176470588235, 1: 10.387031408308005, 2: 13.401307189542484}


In [30]:
num_labels = max(counts.keys()) + 1
weights_tensor = torch.zeros(num_labels, dtype=torch.float)

for label, weight in class_weights.items():
    weights_tensor[label] = weight

print(weights_tensor)

tensor([ 1.2061, 10.3870, 13.4013])


## lấy model 

In [31]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [33]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

In [34]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

id2label = {0:'O', 1:'B-ASP', 2:'I-ASP'}

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids

    # Flatten tất cả token và bỏ padding (-100)
    true_preds = []
    true_labels = []

    for pred_seq, label_seq in zip(preds, labels):
        for p_lab, l_lab in zip(pred_seq, label_seq):
            if l_lab == -100:
                continue
            true_preds.append(p_lab)
            true_labels.append(l_lab)

    # Token-level metrics
    precision = precision_score(true_labels, true_preds, average='macro')
    recall = recall_score(true_labels, true_preds, average='macro')
    f1 = f1_score(true_labels, true_preds, average='macro')

    return {"precision": precision, "recall": recall, "f1": f1}


In [35]:
training_args = TrainingArguments(
    output_dir="./bert_aspect_model",
    num_train_epochs=30,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10, 
    eval_steps=100,               
    eval_strategy="steps",
    save_total_limit=2,
    save_steps=100,
    report_to="none",
    disable_tqdm=False,
    metric_for_best_model="f1",
    greater_is_better=True
)

In [36]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # chuyển class weights sang cùng device với model
        weights = weights_tensor.to(logits.device)

        # CrossEntropyLoss với class weight
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss


In [37]:
from transformers import EarlyStoppingCallback
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

/tmp/ipykernel_19/109506815.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [38]:
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.119800,0.305021,0.865444,0.912041,0.885706
200,0.037700,0.438117,0.870275,0.916820,0.890511
300,0.011400,0.579504,0.904083,0.897494,0.899863
400,0.017900,0.515519,0.885895,0.922698,0.903098
500,0.001200,0.617816,0.899437,0.913636,0.906237
600,0.001400,0.611107,0.904326,0.917053,0.909647
700,0.000400,0.663760,0.909665,0.904042,0.906459
800,0.000800,0.659495,0.906494,0.912729,0.909245
900,0.000200,0.661330,0.903633,0.913407,0.908178
1000,0.000800,0.643852,0.909821,0.919525,0.914329


TrainOutput(global_step=1500, training_loss=0.023496851270397506, metrics={'train_runtime': 290.9703, 'train_samples_per_second': 166.615, 'train_steps_per_second': 5.258, 'total_flos': 1203764213724768.0, 'train_loss': 0.023496851270397506, 'epoch': 29.41176470588235})

In [39]:
trainer.evaluate(eval_dataset=dataset_dict['train'])

{'eval_loss': 3.430464130360633e-05,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 2.5509,
 'eval_samples_per_second': 633.5,
 'eval_steps_per_second': 19.993,
 'epoch': 29.41176470588235}

In [40]:
trainer.evaluate(eval_dataset=dataset_dict['test'])

{'eval_loss': 0.5029063820838928,
 'eval_precision': 0.9171443345406033,
 'eval_recall': 0.9284399086653367,
 'eval_f1': 0.9227154375620636,
 'eval_runtime': 0.3439,
 'eval_samples_per_second': 590.373,
 'eval_steps_per_second': 20.358,
 'epoch': 29.41176470588235}

In [41]:
predictions, labels, _ = trainer.predict(dataset_dict['test'])
predictions = np.argmax(predictions, axis=2)

In [42]:
predictions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
true_labels = []
pred_labels = []
for seq_true, seq_pred in zip(labels, predictions):
    for l_true, l_pred in zip(seq_true, seq_pred):
        if l_true != -100:
            true_labels.append(l_true)
            pred_labels.append(l_pred)

In [44]:
from sklearn.metrics import classification_report
report = classification_report(true_labels, pred_labels, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9859    0.9820    0.9839      3214
           1     0.8932    0.9056    0.8993       360
           2     0.8723    0.8978    0.8849       274

    accuracy                         0.9688      3848
   macro avg     0.9171    0.9284    0.9227      3848
weighted avg     0.9692    0.9688    0.9690      3848



## Test

In [45]:
import torch

def predict_aspect(sentence, model, tokenizer, id2label):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Tokenize và chuyển sang device
    encoding = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu())
    labels = [id2label[pred] for pred in predictions]
    
    return list(zip(tokens, labels))

In [46]:
sentence = "But the staff was so horrible to us."
result = predict_aspect(sentence, model, tokenizer, id2label)

for token, label in result:
    print(f"{token}\t{label}")

[CLS]	O
but	O
the	O
staff	B-ASP
was	O
so	O
horrible	O
to	O
us	O
.	O
[SEP]	O


In [47]:
sentence = "The menu is limited but almost all of the dishes are excellent."
result = predict_aspect(sentence, model, tokenizer, id2label)

for token, label in result:
    print(f"{token}\t{label}")

[CLS]	O
the	O
menu	B-ASP
is	O
limited	O
but	O
almost	O
all	O
of	O
the	O
dishes	B-ASP
are	O
excellent	O
.	O
[SEP]	O


## Lưu 

In [48]:
save_path = "./bert_aspect_model_saved"

model.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)

('./bert_aspect_model_saved/tokenizer_config.json',
 './bert_aspect_model_saved/special_tokens_map.json',
 './bert_aspect_model_saved/vocab.txt',
 './bert_aspect_model_saved/added_tokens.json',
 './bert_aspect_model_saved/tokenizer.json')

In [49]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)


# Đánh giá khía cạnh 

## Tiền xử lý

In [50]:
# /kaggle/input/Restaurants_Test_Data_PhaseB.csv
# /kaggle/input/restaurants-trial.xml
# /kaggle/input/Restaurants_Train_v2.csv
# /kaggle/input/Restaurants_Train_v2.xml
# /kaggle/input/Restaurants_Test_Data_phaseB.xml
# /kaggle/input/Restaurants_Test_Data_PhaseA.csv
# /kaggle/input/Restaurants_Test_Data_PhaseA.xml
# /kaggle/input/Restaurants_Train.xml
# /kaggle/input/restaurants-trial.csv
path_train = "/kaggle/input/Restaurants_Train_v2.csv"
path_trial = "/kaggle/input/restaurants-trial.csv"

In [51]:
df_train = pd.read_csv(path_train)
df_trial = pd.read_csv(path_trial)

In [52]:
df_all = pd.concat([df_train, df_trial], axis=0, ignore_index=True)

In [53]:
df_all = df_all[['Sentence', 'Aspect Term', 'polarity']].copy()

In [54]:
label2id = {"negative":0, "neutral":1, "positive":2,"conflict":3}

In [55]:
id2label = {v:k for k,v in label2id.items()}

In [56]:
df_all['label'] = df_all['polarity'].map(label2id)

In [57]:
df_all.head()

,Sentence,Aspect Term,polarity,label
0,But the staff was so horrible to us.,staff,negative,0
1,"To be completely fair, the only redeeming fact...",food,positive,2
2,"The food is uniformly exceptional, with a very...",food,positive,2
3,"The food is uniformly exceptional, with a very...",kitchen,positive,2
4,"The food is uniformly exceptional, with a very...",menu,neutral,1


In [58]:
print(df_all['label'].isnull().sum())


0


In [59]:
df_all = df_all.dropna(subset=['label'])

In [60]:
df_all.head()

,Sentence,Aspect Term,polarity,label
0,But the staff was so horrible to us.,staff,negative,0
1,"To be completely fair, the only redeeming fact...",food,positive,2
2,"The food is uniformly exceptional, with a very...",food,positive,2
3,"The food is uniformly exceptional, with a very...",kitchen,positive,2
4,"The food is uniformly exceptional, with a very...",menu,neutral,1


In [61]:
df_all['label'] = df_all['label'].astype(int)

In [62]:
label_counts = df_all['label'].value_counts()
print(label_counts)

label
2    2232
0     823
1     643
3      91
Name: count, dtype: int64


In [63]:
all_dataset = Dataset.from_pandas(df_all[['Sentence','Aspect Term','label']])

In [64]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [65]:
def tokenize_fn(batch):
    return tokenizer(batch['Aspect Term'], batch['Sentence'], 
                     truncation=True, padding='max_length', max_length=128)

In [66]:
all_dataset = all_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/3789 [00:00<?, ? examples/s]

In [67]:
all_dataset

Dataset({
    features: ['Sentence', 'Aspect Term', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3789
})

In [68]:
def convert_label_to_int(batch):
    batch['label'] = int(batch['label']) 
    return batch

all_dataset = all_dataset.map(convert_label_to_int)

Map:   0%|          | 0/3789 [00:00<?, ? examples/s]

In [69]:
train_test = all_dataset.train_test_split(test_size=0.2, seed=42)

# từ phần tạm (20%) → tách tiếp thành val (10%) và test (10%)
# 10% trên tổng = 0.5 * 20%
val_test = train_test['test'].train_test_split(test_size=0.5, seed=42)

# gộp lại thành dataset dict
dataset_dict = {
    'train': train_test['train'],
    'validation': val_test['train'],
    'test': val_test['test']
}

In [70]:
dataset_dict

{'train': Dataset({
     features: ['Sentence', 'Aspect Term', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 3031
 }),
 'validation': Dataset({
     features: ['Sentence', 'Aspect Term', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 379
 }),
 'test': Dataset({
     features: ['Sentence', 'Aspect Term', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 379
 })}

In [71]:
print(dataset_dict['train']['Sentence'][0])
print(dataset_dict['train']['Aspect Term'][0])
print(dataset_dict['train']['label'][0])
print(id2label[dataset_dict['train']['label'][0]])
tokens = tokenizer.convert_ids_to_tokens(dataset_dict['train']['input_ids'][0])

tokens_no_pad = [t for t, id_ in zip(tokens, dataset_dict['train']['input_ids'][0]) if id_ != tokenizer.pad_token_id]

print(tokens_no_pad)

Service was excellent, and the AC worked very well too (thank God, it was hot!).
AC
2
positive
['[CLS]', 'ac', '[SEP]', 'service', 'was', 'excellent', ',', 'and', 'the', 'ac', 'worked', 'very', 'well', 'too', '(', 'thank', 'god', ',', 'it', 'was', 'hot', '!', ')', '.', '[SEP]']


## Model

In [72]:
from transformers import AutoModelForSequenceClassification

num_labels = 4
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [74]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

In [75]:
training_args = TrainingArguments(
    output_dir="./bert_sentiment_model",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100, 
    eval_strategy="steps",
    eval_steps=100,              
    save_total_limit=2,
    save_steps=100,
    report_to="none",
    disable_tqdm=False,
    metric_for_best_model="f1",
    greater_is_better=True
)


In [76]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)

    acc = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        p.label_ids, preds, average="weighted"
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [77]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

/tmp/ipykernel_19/3618742650.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [78]:
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.814900,0.775217,0.741425,0.712188,0.741425,0.722389
200,0.545800,0.658478,0.783641,0.767554,0.783641,0.772698
300,0.345800,0.639372,0.791557,0.801235,0.791557,0.792997
400,0.183700,0.785319,0.810026,0.798972,0.810026,0.797838
500,0.110200,0.840889,0.802111,0.821330,0.802111,0.808280
600,0.049800,1.091907,0.786280,0.794172,0.786280,0.784091
700,0.042900,1.113144,0.799472,0.810718,0.799472,0.800825
800,0.019200,1.106730,0.804749,0.809827,0.804749,0.805087
900,0.012700,1.132403,0.807388,0.812043,0.807388,0.807007


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=950, training_loss=0.22410240345879606, metrics={'train_runtime': 410.476, 'train_samples_per_second': 73.841, 'train_steps_per_second': 2.314, 'total_flos': 1993759823677440.0, 'train_loss': 0.22410240345879606, 'epoch': 10.0})

In [79]:
trainer.evaluate(eval_dataset=dataset_dict['validation'])

{'eval_loss': 1.1326833963394165,
 'eval_accuracy': 0.8047493403693932,
 'eval_precision': 0.8043204150418702,
 'eval_recall': 0.8047493403693932,
 'eval_f1': 0.8021461109299816,
 'eval_runtime': 1.3501,
 'eval_samples_per_second': 280.725,
 'eval_steps_per_second': 8.888,
 'epoch': 10.0}

In [80]:
trainer.evaluate(eval_dataset=dataset_dict['test'])

{'eval_loss': 1.230118751525879,
 'eval_accuracy': 0.8021108179419525,
 'eval_precision': 0.7970087429005858,
 'eval_recall': 0.8021108179419525,
 'eval_f1': 0.7961415680441911,
 'eval_runtime': 1.3758,
 'eval_samples_per_second': 275.476,
 'eval_steps_per_second': 8.722,
 'epoch': 10.0}

In [81]:
import numpy as np
from sklearn.metrics import classification_report

def evaluate_model(trainer, test_dataset, id2label):
    """
    trainer: HuggingFace Trainer đã train xong
    test_dataset: HF Dataset
    id2label: dict mapping {0:'negative',1:'neutral',2:'positive'}
    """
    # Lấy prediction trực tiếp từ trainer
    preds_output = trainer.predict(test_dataset)
    
    y_true = preds_output.label_ids
    y_pred = np.argmax(preds_output.predictions, axis=1)

    # In báo cáo chi tiết
    report = classification_report(
        [id2label[i] for i in y_true],
        [id2label[i] for i in y_pred],
        digits=4
    )
    print(report)
    return report


In [82]:
id2label = {0:'negative', 1:'neutral', 2:'positive',3:'conflict'}
report = evaluate_model(trainer, dataset_dict['test'], id2label)


              precision    recall  f1-score   support

    conflict     0.6667    0.2857    0.4000        14
    negative     0.7162    0.7162    0.7162        74
     neutral     0.7458    0.6875    0.7154        64
    positive     0.8458    0.8943    0.8694       227

    accuracy                         0.8021       379
   macro avg     0.7436    0.6459    0.6753       379
weighted avg     0.7970    0.8021    0.7961       379



## Test

In [83]:
import torch

def predict_aspect_sentiment(aspect, sentence, tokenizer, model, max_length=128):
    """
    aspect: str, từ/cụm từ aspect
    sentence: str, câu chứa aspect
    tokenizer: BERT tokenizer
    model: trained AutoModelForSequenceClassification
    id2label: dict mapping {0: 'negative', 1: 'neutral', 2: 'positive',3:'conflict'}
    """
    model.eval()
    
    inputs = tokenizer(
        aspect,
        sentence,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'  
    )

    device = next(model.parameters()).device
    inputs = {k:v.to(device) for k,v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # [1, 3]
        pred_id = torch.argmax(logits, dim=-1).item() 
    
    return pred_id


In [84]:
id2label = {0:'negative', 1:'neutral', 2:'positive',3:'conflict'}

In [85]:
aspect = "service"
sentence = "The food was great but the service was very slow."
pred = predict_aspect_sentiment(aspect, sentence, tokenizer, model)

In [86]:
print(id2label[pred])

negative


In [87]:
aspect = "food"
sentence = "The pasta was fresh and tasty, but the price was too high."
pred = predict_aspect_sentiment(aspect, sentence, tokenizer, model)
print(id2label[pred])

neutral


## Lưu

In [88]:
save_path = "./bert_sentiment_model_saved"

model.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)

('./bert_sentiment_model_saved/tokenizer_config.json',
 './bert_sentiment_model_saved/special_tokens_map.json',
 './bert_sentiment_model_saved/vocab.txt',
 './bert_sentiment_model_saved/added_tokens.json',
 './bert_sentiment_model_saved/tokenizer.json')